<a href="https://colab.research.google.com/github/chonholee/tutorial/blob/main/bigdata/BigDataII_11_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
%cd '/content/drive/MyDrive/Lecture_BigData'

In [ ]:
import tensorflow as tf

# CNN (Convolutional Neural Network)

In [ ]:
# (1) 手書き数字画像のデータセットをダウンロード・正規化
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

num_classes = 10

channel = 1

In [ ]:
# (2) 簡単なCNNモデルを構築してみる

from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Flatten, Dense, Dropout

model = tf.keras.models.Sequential()
model.add( Conv2D(32, (3, 3), input_shape=(28, 28, channel), activation='relu') )      # 追加
model.add( MaxPooling2D(pool_size=(2,2)) )              # 追加
model.add( Conv2D(32, (3, 3), activation='relu') )      # 追加
model.add( MaxPooling2D(pool_size=(2,2)) )              # 追加
model.add( Flatten() )
model.add( Dense(128, activation='relu') )
model.add( Dropout(0.2) )
model.add( Dense(num_classes, activation='softmax') )

# モデルの構造を見てみる
model.summary()

In [ ]:
# (3) モデルのコンパイル
model.compile(optimizer='Adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

# (4) モデルの学習開始
model.fit(x_train, y_train, epochs=5)

In [ ]:
# (5) モデルの評価
model.evaluate(x_test,  y_test, verbose=2)

In [ ]:
# 保存
model_filename = 'mymodel_cnn'
model.save(model_filename)

# cifar10 Dataset (一般カラー画像) の分類

*runtimeの種類をGPUに変更することを勧めます。学習の処理が早くなります。*

In [ ]:
cifar10 = tf.keras.datasets.cifar10

# データをロードして、x_train, y_train, x_test, y_testを準備してください

here

In [ ]:
# データのサイズ（shape）、画像1枚のサイズ（縦横ピクセルサイズ）を確認してください

here

In [ ]:
# RGB画像、R（レッド）,G（グリーン）,B（ブルー）の三つのチャンネルにそれぞれ0から255の値が入っています。
# その値を255.0で割ることによって、R,G,Bに入っている値が0から1までになるようにしてください。

here

In [ ]:
# 好きな画像を10枚、ラベルと一緒に表示させなさい。

here


In [ ]:
# CNNモデルを定義しなさい
#   基本的にはMNISTデータの演習で行ったCNNと同じ構造で試してみましょう。
#   入力データのサイズとチャネル数、クラスの数を確認して修正

here

In [ ]:
# モデルのコンパイル

here

In [ ]:
# モデルの学習
#   epochs=5 で学習

here

In [ ]:
# モデルの評価

here

In [ ]:
# model.predict(...)、numpy.argmax(...) などを使って、test画像をいくつか分類してみてください。

here

In [ ]:
# モデルを再学習
#   epochs=10 で学習

here

In [ ]:
# モデルの評価
# lossは下がってますか？accuracyはあがってますか？

here

In [ ]:
# 上で間違って分類されたtest画像をいくつか分類してみてください。

here

# 自前のデータセットを用意して、異なる分類モデルを学習させてみよう

前準備

* 学習データセット用のフォルダを用意する
* 分類ラベルごとの**サブフォルダ**を用意する
* 各サブフォルダに、画像を保存する

今回は例として猫と犬の画像を分類する

* 「Cat」「Dog」のサブフォルダを用意して画像を保存する

In [ ]:
directory = "petimages-subset/"

In [ ]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

BATCH_SIZE = 32
IMG_SIZE = (224, 224)
train_dataset = image_dataset_from_directory(directory,
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE,
                                             validation_split=0.2,
                                             subset='training',
                                             seed=42)
validation_dataset = image_dataset_from_directory(directory,
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE,
                                             validation_split=0.2,
                                             subset='validation',
                                             seed=42)

In [ ]:
class_names = train_dataset.class_names
print(class_names)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title("{}: {}".format(labels[i].numpy(), class_names[labels[i]]))
        plt.axis("off")

In [ ]:
# 正規化 (別の書き方)

norm_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1/255.)

norm_train_dataset = train_dataset.map(lambda x, y: (norm_layer(x), y))
norm_val_dataset = validation_dataset.map(lambda x, y: (norm_layer(x), y))

## VGG16 モデル

imagenetをベースとしたCNNモデルの一種

In [ ]:
# データの拡張Layerを定義

# 学習データが少ない時、現状で用意できるデータセットからデータ数を増やす手段
#    参照：https://www.codexa.net/data_augmentation_python_keras/

data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.RandomRotation(0.2),
])

In [ ]:
#--- Layerの準備 ---

IMG_SIZE = (32, 32, 3)

# 入力の前処理として「データ拡張」を行う
inputs = tf.keras.Input(shape=(None, None, 3))
x = data_augmentation(inputs)
x = tf.keras.layers.Lambda(lambda img: tf.image.resize(img, (IMG_SIZE[0], IMG_SIZE[1])))(x)

In [ ]:
""" VGG16 """
x = tf.keras.layers.Lambda(tf.keras.applications.vgg16.preprocess_input)(x)

base_model = tf.keras.applications.vgg16.VGG16(
      input_shape=IMG_SIZE,
      input_tensor=x,
      include_top=False, 
      weights='imagenet'     
)

base_model.trainable = True # base model の重みを固定する場合

# 全結合層出なくGlobal Average Pooling を使用することで計算量を減らせます
GAP_layer = GlobalAveragePooling2D()

# 最終層
pred_layer = Dense(len(class_names), activation='softmax')

#--- モデルの構築 ---
model = tf.keras.Sequential([
    base_model,
    GAP_layer,
    pred_layer
])

In [ ]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 3, 3, 512)         14714688  
                                                                 
 global_average_pooling2d_3   (None, 512)              0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_5 (Dense)             (None, 2)                 1026      
                                                                 
Total params: 14,715,714
Trainable params: 14,715,714
Non-trainable params: 0
_________________________________________________________________


In [ ]:
base_model.summary()

In [ ]:
# (3) モデルのコンパイル
model.compile(optimizer='Adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

# (4) モデルの学習開始
model.fit(norm_train_dataset, validation_data=norm_val_dataset, epochs=3)

In [ ]:
# 保存
model_filename = 'mymodel/vgg16'
model.save_weights(model_filename)

In [ ]:
# 読み込み
model.load_weights(model_filename)

In [ ]:
# 再学習
model.fit(norm_train_dataset, validation_data=norm_val_dataset, epochs=3)

In [ ]:
# 保存
model_filename = 'mymodel/vgg16'
model.save_weights(model_filename)

In [ ]:
import numpy as np

for images, labels in validation_dataset.take(1): # <--- テスト用にValidation画像を用いる

    # テスト画像を一度に分類できる
    imgs = tf.image.resize(images, (IMG_SIZE[0], IMG_SIZE[1]))
    outputs = model.predict(imgs) # softmax の結果（信頼度）
    predicted = np.argmax(outputs, 1)
    print(outputs)
    print(predicted)

In [ ]:
# 結果を一枚ずつ確認
for i in range(len(outputs)):
    print('test', i, end=':')
    print(' predicted class label', class_names[predicted[i]], end=',')
    print(' true class label', class_names[labels[i]])

## MobileNet V2

デバイスでも動くようにパラメータの数を減らして最適化したモデル

In [ ]:
#--- Layerの準備 ---
num_classes = 2

IMG_SIZE = (96, 96, 3)

# 入力の前処理
inputs = tf.keras.Input(shape=(None, None, 3))
x = data_augmentation(inputs)
x = tf.keras.layers.Lambda(lambda img: tf.image.resize(img, (IMG_SIZE[0], IMG_SIZE[1])))(x)
x = tf.keras.layers.Lambda(tf.keras.applications.mobilenet_v2.preprocess_input)(x)

base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMG_SIZE,
    input_tensor=x,
    include_top = False,  #False にすることで出力層は読み込みません
    weights='imagenet')

base_model.trainable = True # base model の重みを固定する場合

# 全結合層出なくGlobal Average Pooling を使用することで計算量を減らせます
GAP_layer = GlobalAveragePooling2D()

# 最終層
pred_layer = Dense(num_classes, activation='softmax')

#--- モデルの構築 ---
model = tf.keras.Sequential([
    base_model,
    GAP_layer,
    pred_layer
])

In [ ]:
base_model.summary()

In [ ]:
# petimages用
model.compile(optimizer='Adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.fit(norm_train_dataset, validation_data=norm_val_dataset, epochs=5)

In [ ]:
# 保存
model.save_weights("mymodel/mobilenetv2-petimage")

In [ ]:
# cifar10用
cifar10 = tf.keras.datasets.cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
num_classes = 10

model.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=5)

In [ ]:
model.save_weights("mymodel/mobilenetv2-cifar10")

## ResNet50

モデルの層をより深く

参照：https://deepage.net/deep_learning/2016/11/30/resnet.html

※※※

モデル構築の部分を少し違った描き方で書いています。

VGGとMobileNetのサンプルでは、層を定義して最後にまとめていますが

以下ResNetのサンプルでは、層を定義すると同時にモデルの構築を行っています。

In [ ]:
from tensorflow.keras.layers import Input, Add, Dense, Activation, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, Dropout
from tensorflow.keras import Model
from tensorflow.keras.applications import ResNet50

IMG_SIZE = (224, 224, 3)

def get_model():
    base_model_res50 = ResNet50(
         include_top=True, 
         input_shape=IMG_SIZE,
         weights='imagenet')
    # take the last global average pooling with fewer parameters
    x = base_model_res50.layers[-2].output
    
    x = Dense(2048)(x)
    x = Activation('relu')(x)
    x = Dropout(.5)(x)
    
    x = Dense(2048)(x)
    x = Activation('relu')(x)
    x = Dropout(.5)(x)
    
    x = Dense(10)(x)
    outputs = Activation('softmax')(x)

    model = Model(base_model_res50.input, outputs)
    return model

In [ ]:
model = get_model()

model.summary()

In [ ]:
model.compile(optimizer='Adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

model.fit(norm_train_dataset, validation_data=norm_val_dataset, epochs=5)

# 補足（時間の関係で講義内に説明しきればかった内容）

オンラインの資料などを見ながら勉強してみてください。

*   **softmax**
*   **one-hot encoding**

今回の演習や課題で扱ったデータセットのラベルはカテゴリーデータ（離散値）です。

*   mnist dataでは、０～９までの数字
*   cifar10 dataでは、物体の名前に対応した0～９までの数字

例えば、y_train[0] には（x_train[0]に対応した）１つの値が格納されています。

ですが、モデルの最終層のニューロンの数はクラスの数（10個）となっています。

```
model.add( tf.keras.layers.Dense(10, activation='softmax') )
```

講義で損失関数（真値と計算値の誤差）の話をしましたが、現在真値は1個、計算値は10個の値があるので単純に引き算（損失関数）を計算できません。

実は
```
model.compile(optimizer='Adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
```
の「sparse categorical crossentoropy」内で上手く計算してくれています。

一般的には、真値の数を最終層の数（分類するクラスの数）と合わせるための処理をします。

その一つが 「**one-hot encodings**」 と呼ばれるものです。

例えば

2 (bird) は [0,0,1,0,0,0,0,0,0,0]

7 (horse) は [0,0,0,0,0,0,0,1,0,0]

とエンコーディング変換する事を言います。

In [ ]:
import numpy as np

print(y_train[0]) # 確認

y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

print(y_train[0]) # 確認

クラスラベル（y_train と y_test）を one-hot として扱う場合は、「categorical_crossentropy」というlossを使います。

試してみてください。

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

In [ ]:
a = [[('n04592741', 'wing', 0.9714641), ('n02690373', 'airliner', 0.025861548), ('n04266014', 'space_shuttle', 0.0013809769)], [('n03899768', 'patio', 0.51726824), ('n03028079', 'church', 0.25273263), ('n04005630', 'prison', 0.046136115)], [('n02129604', 'tiger', 0.767321), ('n02123159', 'tiger_cat', 0.22703454), ('n02391049', 'zebra', 0.004188579)], [('n02504013', 'Indian_elephant', 0.7616036), ('n01871265', 'tusker', 0.21664836), ('n02504458', 'African_elephant', 0.021731125)], [('n11939491', 'daisy', 0.26833987), ('n01828970', 'bee_eater', 0.14348479), ('n03876231', 'paintbrush', 0.08295382)]]

In [ ]:
for i, labels in enumerate(a):
  print(i, labels)

0 [('n04592741', 'wing', 0.9714641), ('n02690373', 'airliner', 0.025861548), ('n04266014', 'space_shuttle', 0.0013809769)]
1 [('n03899768', 'patio', 0.51726824), ('n03028079', 'church', 0.25273263), ('n04005630', 'prison', 0.046136115)]
2 [('n02129604', 'tiger', 0.767321), ('n02123159', 'tiger_cat', 0.22703454), ('n02391049', 'zebra', 0.004188579)]
3 [('n02504013', 'Indian_elephant', 0.7616036), ('n01871265', 'tusker', 0.21664836), ('n02504458', 'African_elephant', 0.021731125)]
4 [('n11939491', 'daisy', 0.26833987), ('n01828970', 'bee_eater', 0.14348479), ('n03876231', 'paintbrush', 0.08295382)]
